## Working at 95% accuracy, model weights output and test image shown

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Preprocess the data
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define the binary neural network model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, input_shape=(784,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Define the binary weights and activations
binary_model = tf.keras.models.clone_model(model)
binary_weights = binary_model.get_weights()
for i in range(len(binary_weights)):
    binary_weights[i] = np.where(binary_weights[i] >= 0, 1, -1)
binary_model.set_weights(binary_weights)

# Compile the model
binary_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = binary_model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Print the accuracy of the model
print("Test accuracy:", history.history['val_accuracy'][-1])

# Use the model to predict the digit for a test image
test_image = x_test[1]
test_image = np.expand_dims(test_image, axis=0)
predicted_digit = np.argmax(binary_model.predict(test_image))


print("Image Found:")
image=x_test[1]
label = y_test[1]
plt.imshow(image.reshape(28,28), cmap='gray')
plt.title('Label: ' + str(label))
plt.show()


print("Predicted digit:", predicted_digit)

In [ ]:
image_num = 0
test_image = x_test[image_num]
label = y_test[image_num]
test_image = np.expand_dims(test_image, axis=0)
predicted_digit = np.argmax(binary_model.predict(test_image))

print("Image Found:")
plt.imshow(test_image.reshape(28,28), cmap='gray')
plt.title('Label: ' + str(label))
plt.show()

print("Predicted digit:", predicted_digit)

In [ ]:
# Print the binary weights of the model
for layer in binary_model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        binary_weights = layer.get_weights()
        binary_weights[0] = np.where(binary_weights[0] >= 0, 1, -1) # change the -1 to a 0 if you need the 0/1 binarized
        layer.set_weights(binary_weights)
        print(layer.get_weights()[0]) # gets the binarized output array at each layer
        print(layer.get_weights()[0].shape) # prints the dimensions of the matrix

In [ ]:
# Loop through the layers to find the BatchNormalization layers
for layer in binary_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):

        # Get the epsilon value of the layer
        epsilon = layer.epsilon

        # Get the gamma value of the layer
        gamma = layer.gamma.numpy()

        # Get the beta value of the layer
        beta = layer.beta.numpy()

        # Print the values
        print("Epsilon:", epsilon)
        print("Gamma:", gamma,"size:",gamma.shape)
        print("Beta:", beta,"size:",beta.shape)


In [ ]:
# Compute the density matrix and save to MIF
layer_num = 0
for layer in binary_model.layers:
    if isinstance(layer, tf.keras.layers.Dense):
        binary_weights = layer.get_weights()
        binary_weights[0] = np.where(binary_weights[0] >= 0, 1, 0) # change the second 0 to -1 for true binarized matrix
        layer.set_weights(binary_weights)
        density_matrix = np.matmul(binary_weights[0], binary_weights[0].T) / binary_weights[0].shape[1]

        # Save the density matrix to a mif file
        with open(f'density_matrix_{layer_num}.mif', 'w') as f:
            f.write('DEPTH = {}\n'.format(density_matrix.size))
            f.write('WIDTH = 1\n')
            f.write('ADDRESS_RADIX = DEC\n')
            f.write('DATA_RADIX = HEX\n')
            f.write('CONTENT\n')
            f.write('BEGIN\n')
            address = 0
            for i in range(density_matrix.shape[0]):
                for j in range(density_matrix.shape[1]):
                    hex_value = hex(int(density_matrix[i,j]*1024))[2:].zfill(4)
                    for bit in range(len(hex_value)*4):
                        bit_value = int(hex_value, 16) >> (len(hex_value)*4 - bit - 1) & 1
                        f.write('{} : {}\n'.format(address, bit_value))
                        # print('{} : {}\n'.format(address, bit_value), end = ' ')
                        address += 1
                # print("")
            f.write('END;\n')

        layer_num += 1


In [ ]:
# Get the gamma, beta, and epsilon matrices from the model nad store in MIF
layer_num = 0
gamma, beta, epsilon = [], [], []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer_num+=1
        gamma=[]
        beta=[]
        gamma.append(layer.gamma.numpy())
        beta.append(layer.beta.numpy())
        epsilon=layer.epsilon

        # Write gamma to a MIF file
        with open(f'gamma_{layer_num}.mif', 'w') as f:
            f.write('WIDTH = %d;\n' % gamma[0].shape[0])
            f.write('DEPTH = %d;\n' % 1)
            f.write('ADDRESS_RADIX = HEX;\n')
            f.write('DATA_RADIX = HEX;\n')
            f.write('CONTENT\n')
            f.write('BEGIN\n')
            for j in range(gamma[0].shape[0]):
                f.write('%X : %X;\n' % (i * gamma[0].shape[0] + j, np.uint16(gamma[0][j] * (2 ** 15 - 1))))
            f.write('END;\n')

        # Write beta to a MIF file
        with open(f'beta_{layer_num}.mif', 'w') as f:
            f.write('WIDTH = %d;\n' % beta[0].shape[0])
            f.write('DEPTH = %d;\n' % 1)
            f.write('ADDRESS_RADIX = HEX;\n')
            f.write('DATA_RADIX = HEX;\n')
            f.write('CONTENT\n')
            f.write('BEGIN\n')
            for j in range(beta[0].shape[0]):
                f.write('%X : %X;\n' % (i * beta[0].shape[0] + j, np.uint16(beta[0][j] * (2 ** 15 - 1))))
            f.write('END;\n')

        # Write epsilon to a MIF file
        with open(f'epsilon_{layer_num}.mif', 'w') as f:
            f.write('WIDTH = %d;\n' % 32)
            f.write('DEPTH = %d;\n' % 1)
            f.write('ADDRESS_RADIX = HEX;\n')
            f.write('DATA_RADIX = HEX;\n')
            f.write('CONTENT\n')
            f.write('BEGIN\n')
            f.write('0 : %X;\n' % np.uint32(epsilon * (2 ** 15 - 1)))
            f.write('END;\n')


In [ ]:
# Extracting 10 MNIST dataset images to test our FPGA-Based Model
import numpy as np

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Select 10 images
images = x_train[:10]

# Convert each image to MIF format
for i, img in enumerate(images):
    # Reshape image to 1D array
    img = img.reshape(-1)

    # Convert pixel values to hexadecimal
    hex_values = ['{:02X}'.format(int(p * 255)) for p in img]

    # Write MIF file
    with open(f'image_{i}.mif', 'w') as f:
        f.write('WIDTH=8;\n')
        f.write('DEPTH=784;\n')
        f.write('ADDRESS_RADIX=UNS;\n')
        f.write('DATA_RADIX=HEX;\n')
        f.write('CONTENT BEGIN\n')
        for j, hex_val in enumerate(hex_values):
            f.write(f'{j} : {hex_val};\n') # each address has its own pixel
        f.write('END;\n')